In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tamil-wiki-summarization/Tamil wiki-data.csv
/kaggle/input/tamil-wiki-summarization/Tamil wiki-data summarized.csv


In [16]:
import pandas as pd

df=pd.read_csv("Tamil wiki-data summarized.csv")

In [29]:
df=df.dropna()

In [18]:
from transformers import AutoTokenizer

In [19]:
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")

Downloading:   0%|          | 0.00/507 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.38M [00:00<?, ?B/s]

In [40]:
max_input_length = 10000
max_target_length = 128


def preprocess_function(examples):
    inputs = [doc for doc in examples["Original"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["Summary"], max_length=max_target_length, truncation=True
        )

    return model_inputs

In [44]:
from datasets import Dataset

dataset = Dataset.from_pandas(df)

In [45]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)

  0%|          | 0/157 [00:00<?, ?ba/s]

In [37]:
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM

# You can of course substitute your own username and model here if you've trained and uploaded it!
model_name = 'ai4bharat/IndicBARTSS'
tokenizer = AutoTokenizer.from_pretrained(model_name,from_pt=True)
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_name,from_pt=True)

Downloading:   0%|          | 0.00/931M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFMBartForConditionalGeneration: ['lm_head.weight']
- This IS expected if you are initializing TFMBartForConditionalGeneration from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFMBartForConditionalGeneration from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFMBartForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMBartForConditionalGeneration for predictions without further training.


In [48]:
document="அங்கிருந்து அழுது கொண்டே வீட்டிற்கு சென்ற சிறுமி தன் பெற்றோரிடம் இது குறித்து தகவல் தெரிவிக்கவே அதிர்ச்சிக்குள்ளான பெற்றோர்கள் வேப்பங்குப்பம் காவல் நிலையத்தில் இரவு 12 மணி அளவில் தகவல் தெரிவித்துள்ளனர். புகாரின் அடிப்படையில் வேப்பங்குப்பம் போலீசார் சரத்குமாரை கைது செய்து காவல் நிலையம் அழைத்து வந்து விசாரணை செய்து சிறையில் அடைத்தனர்."
tokenized = tokenizer([document], return_tensors='np')
out = model.generate(**tokenized, max_length=128)